In [24]:
import pandas as pd
import numpy as np

df = pd.read_parquet('baseline_raw.parquet')

In [25]:
df['foto_mes'] = pd.to_datetime(df['foto_mes'], format='%Y%m')

In [26]:
df_val = df[df['foto_mes']=='2021-07-01']
df_train = df[df['foto_mes']<'2021-07-01']

In [27]:
b1_clients = pd.read_csv('B1_clients_202107.csv')
b1 = set(b1_clients['numero_de_cliente'])
df_val['target'] = np.where(df_val['numero_de_cliente'].isin(b1), 'B1', df_val['target'])

In [29]:
df_val.shape, df_train.shape

((164682, 155), (979558, 155))

In [30]:
# X_train = df_train.drop(columns=['target'])
# X_val = df_val.drop(columns=['target'])
# y_train = df_train['target']
# y_val = df_val['target']

In [31]:
X_val = df_val.drop(columns=['target'])
y_val = df_val['target']

In [32]:
%%time
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(df_train)
val_data = TabularDataset(df_val)

CPU times: user 9.2 ms, sys: 40.7 ms, total: 49.9 ms
Wall time: 49.8 ms


In [33]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [15]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column="numero_de_cliente",
    timestamp_column="foto_mes"
)
train_data.head()

,,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,target
item_id,timestamp,,,,,,,,,,,,,,,,,,,,,
29183981,2021-01-01,True,False,0,50,195,16177.759766,98482.671875,3389.110107,8038.750000,2863.770020,...,11.0,167739.000000,-52785.000000,0.00,5879.0,85557.031250,75.0,0.0,73511.906250,C
29184630,2021-01-01,True,False,0,59,320,30764.570312,68988.710938,32456.330078,-4944.830078,3502.580078,...,4.0,0.000000,-56265.808594,1.24,4030.0,32056.880859,8.0,0.0,2604.060059,C
29185433,2021-01-01,True,False,0,67,266,692.190002,24121.130859,501.829987,-1278.089966,1440.380005,...,25.0,3049.800049,-3049.800049,0.00,7770.0,1236.069946,2.0,0.0,891.479980,C
29185587,2021-01-01,True,False,0,79,320,6934.649902,45318.511719,6645.339844,-624.309998,870.950012,...,4.0,0.000000,-23249.009766,0.00,7521.0,15789.360352,9.0,0.0,997.049988,C
29185646,2021-01-01,True,False,0,60,255,6576.720215,6285.450195,2184.520020,-2153.179932,5897.600098,...,25.0,87595.546875,-151980.265625,4.34,7768.0,80493.859375,27.0,0.0,5020.439941,C


In [16]:
val_data = TimeSeriesDataFrame.from_data_frame(
    df_val,
    id_column="numero_de_cliente",
    timestamp_column="foto_mes"
)
val_data.head()

,,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,target
item_id,timestamp,,,,,,,,,,,,,,,,,,,,,
29183981,2021-07-01,True,False,0,50,201,13073.360352,161035.984375,3163.520020,5554.229980,2894.060059,...,10.0,0.00000,-82110.0000,0.00,6060.0,121585.390625,75.0,0.0,77617.406250,C
29184630,2021-07-01,True,False,0,59,326,2640.479980,56751.511719,5232.220215,-4483.750000,2274.250000,...,3.0,0.00000,-85347.9375,17.59,4211.0,21596.400391,7.0,0.0,4938.330078,C
29185433,2021-07-01,True,False,0,68,272,1343.510010,20201.310547,582.940002,-780.789978,1429.189941,...,24.0,4692.00000,-4692.0000,0.00,7951.0,1529.020020,2.0,0.0,1407.599976,C
29185587,2021-07-01,True,False,0,80,326,5523.189941,55827.281250,2712.260010,-498.619995,2894.979980,...,3.0,0.00000,-26420.0000,0.00,7702.0,12815.990234,8.0,0.0,856.289978,C
29185646,2021-07-01,True,False,0,60,261,4819.350098,37386.398438,1878.400024,-4232.870117,6740.069824,...,24.0,174063.71875,-160642.1250,12.89,7949.0,157186.125000,27.0,0.0,9548.219727,C


In [17]:
help(TabularPredictor.fit)

Help on function fit in module autogluon.tabular.predictor.predictor:

fit(self, train_data, tuning_data=None, time_limit=None, presets=None, hyperparameters=None, feature_metadata='infer', infer_limit=None, infer_limit_batch_size=None, fit_weighted_ensemble=True, calibrate_decision_threshold=False, num_cpus='auto', num_gpus='auto', **kwargs)
    Fit models to predict a column of a data table (label) based on the other columns (features).
    
    Parameters
    ----------
    train_data : str or :class:`TabularDataset` or :class:`pd.DataFrame`
        Table of the training data, which is similar to a pandas DataFrame.
        If str is passed, `train_data` will be loaded using the str value as the file path.
    tuning_data : str or :class:`TabularDataset` or :class:`pd.DataFrame`, default = None
        Another dataset containing validation data reserved for tuning processes such as early stopping and hyperparameter tuning.
        This dataset should be in the same format as `train_

In [8]:
predictor = TabularPredictor(label='target', eval_metric='f1').fit(
    train_data,
    time_limit=6*3600,
    num_cpus=24,
    excluded_model_types = ['KNN', 'XT' ,'RF'],
    num_stack_levels=0,
    fit_weighted_ensemble=False
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231119_235129/"
Beginning AutoGluon training ... Time limit = 21600s
AutoGluon will save models to "AutogluonModels/ag-20231119_235129/"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #85-Ubuntu SMP Fri Jul 7 15:25:09 UTC 2023
Disk Space Avail:   311.21 GB / 982.70 GB (31.7%)
Train Data Rows:    979558
Train Data Columns: 154
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['C', 'B2']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = C, class 0 = B2
	Note: For your binary classification, AutoGluon arbitrarily selected which label-

In [36]:
predictor = TabularPredictor.load("AutogluonModels/ag-20231119_235129/")
preds = predictor.predict_proba(X_val)
preds = preds['B2']
y_val = np.where(y_val=='B2', 1, 0)

In [37]:
pd.Series(y_val).value_counts()

0    163414
1      1268
dtype: int64

In [38]:
def ganancia_integral(probs: np.array, 
                      y_true: np.array) -> float:
    ganancia_df = pd.DataFrame({'prob': probs, 'truth': y_true})
    ganancia_df = ganancia_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    ganancia_df['ganancia_ind'] = np.where(ganancia_df['truth']==1, 273, -7)
    ganancia_df['ganancia_roll'] = ganancia_df['ganancia_ind'].cumsum()
    gan_max = ganancia_df['ganancia_roll'].max()
    ind = ganancia_df[ganancia_df['ganancia_roll']==gan_max].index[0]
    print(ind)
    res = np.mean(ganancia_df['ganancia_roll'].iloc[ind-1000:ind+1001])
    return res

In [39]:
# True value
ganancia_integral(preds, y_val)

15755


123543.96201899051